# VacationPy
----


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("..\WeatherPy\output_data\cities.csv")
city_data

,Unnamed: 0,City Number,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,3139597,Sistranda,63.7256,8.8340,55.99,100,75,5.82,NO,1595823246
1,1,6201424,Mataura,-46.1927,168.8643,50.47,62,0,1.14,NZ,1595819783
2,2,1507390,Dikson,73.5069,80.5464,38.44,87,96,14.05,RU,1595822999
3,3,3451138,Rio Grande,-32.0350,-52.0986,46.00,96,35,9.66,BR,1595823277
4,4,2082539,Merauke,-8.4667,140.3333,81.45,75,57,19.66,ID,1595823063
...,...,...,...,...,...,...,...,...,...,...,...
520,520,581049,Arkhangel’sk,64.5401,40.5433,64.40,88,40,4.47,RU,1595822962
521,521,3662927,Pauini,-7.7136,-66.9764,73.26,70,79,1.45,BR,1595823295
522,522,371745,Kutum,14.2000,24.6667,76.19,49,81,2.53,SD,1595822953
523,523,2077895,Alice Springs,-23.7000,133.8833,64.40,39,0,10.29,AU,1595823063


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Customize the size of the figure
figure_layout = {
    "width": "600px",
    "height": "500px",
    "border": "1px solid black",
    "padding": "1px",
    "margin": "0 auto 0 auto"
}
fig = gmaps.figure(layout=figure_layout)

# Assign the heatmap layer to a variable
locations = city_data[["Lat","Lng"]].to_records(index=False)
humidity = city_data["Humidity"].tolist()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

# Add the heatmap layer to the map
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
dream_vaycay = city_data[(city_data["Max Temp"]>70) & (city_data["Max Temp"]<80)
                & (city_data["Wind Speed"]<10)
                & (city_data["Cloudiness"]==0)]
dream_vaycay = dream_vaycay.drop(["Unnamed: 0"], axis=1) # Drop redundant column
len(dream_vaycay)

8

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df
hotel_df = dream_vaycay

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City Number,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
94,3372964,Lagoa,39.0500,-27.9833,72.28,93,0,6.91,PT,1595823272,
113,2074865,Carnarvon,-24.8667,113.6333,71.60,40,0,5.82,AU,1595823063,
122,2214846,Mişrātah,32.3754,15.0925,76.95,77,0,5.86,LY,1595823075,
157,5983626,Invermere,50.5167,-116.0354,71.01,41,0,2.17,CA,1595823383,
244,2522875,Trapani,37.8333,12.6667,71.01,72,0,4.70,IT,1595823090,
283,2127202,Anadyr,64.7500,177.4833,78.80,57,0,6.71,RU,1595823064,
477,2525041,Cefalù,38.0386,14.0229,73.00,81,0,1.99,IT,1595823091,
508,7341918,Moba,33.2288,105.0179,79.75,39,0,1.32,CN,1595823478,


In [7]:
# Set parameters to search for hotels with 5000 meters
locations = hotel_df[["Lat","Lng"]].to_records(index=False)
target_type = "lodging"
target_radius = 5000

# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Function to query API and retrive results in json
def places_search(locations):
    query_url = f"{base_url}?location={locations[0]},{locations[1]}&radius={target_radius}&type={target_type}&key={g_key}" 
    response = requests.get(query_url).json()
    return response

# Create lists to hold relevant data in loop
hotels = list()

# Loop to run API search
for location in locations:
    
    try:
        response = places_search(location)
        hotels.append(response["results"][0]["name"])
    except:
        hotels.append("No hotels found within 5000 meters radius")

In [8]:
# Add first hotel result to hotel_df DataFrame
hotel_df["Hotel Name"] = hotels
hotel_df = hotel_df.rename(columns={"City Name": "City"})
hotel_df = hotel_df.reset_index()
hotel_df

,index,City Number,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,94,3372964,Lagoa,39.0500,-27.9833,72.28,93,0,6.91,PT,1595823272,Casa Das Faias
1,113,2074865,Carnarvon,-24.8667,113.6333,71.60,40,0,5.82,AU,1595823063,Hospitality Carnarvon
2,122,2214846,Mişrātah,32.3754,15.0925,76.95,77,0,5.86,LY,1595823075,فندق الماسة
3,157,5983626,Invermere,50.5167,-116.0354,71.01,41,0,2.17,CA,1595823383,Copper Point Resort
4,244,2522875,Trapani,37.8333,12.6667,71.01,72,0,4.70,IT,1595823090,No hotels found within 5000 meters radius
5,283,2127202,Anadyr,64.7500,177.4833,78.80,57,0,6.71,RU,1595823064,"Гостиница ""Анадырь"""
6,477,2525041,Cefalù,38.0386,14.0229,73.00,81,0,1.99,IT,1595823091,Hotel La Giara
7,508,7341918,Moba,33.2288,105.0179,79.75,39,0,1.32,CN,1595823478,No hotels found within 5000 meters radius


There are 2 locations (Trapani & Moba) where there are no hotels found within 5000 meters radius on Google API.

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations
                                  ,info_box_content=hotel_info)


# Add the marker layer to the map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…